#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   (quando tiver)


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}

In [ ]:
import pandas as pd

df = pd.read_excel('../produtos.xlsx')

comissao_percentual = 0.185
taxa_transacao_percentual = 0.01
taxa_fixa_padrao = 4
margem_lucro_desejada = 0.20

def calcular_preco_venda(preco_custo):

    if preco_custo <= 8:
        taxa_aplicada = preco_custo / 2
    else:
        taxa_aplicada = taxa_fixa_padrao

    total_percentuais = comissao_percentual + taxa_transacao_percentual + margem_lucro_desejada

    preco_venda = (preco_custo + taxa_aplicada) / (1 - total_percentuais)

    comissao = preco_venda * comissao_percentual
    taxa_transacao = preco_venda * taxa_transacao_percentual
    total_taxas = comissao + taxa_aplicada + taxa_transacao

    lucro = preco_venda - preco_custo - total_taxas

    return pd.Series({
        'Preço Final': round(preco_venda, 2),
        'Taxas': round(total_taxas, 2),
        'Lucro': round(lucro, 2)
    })

produtos_completos = df.copy()
produtos_completos.loc[:, ['Preço Final', 'Taxas', 'Lucro']] = (
    produtos_completos['Valor_unitário'].apply(calcular_preco_venda)
)

produtos_completos

In [ ]:
import pandas as pd

df = pd.read_excel('/home/lucas-silva/auto_shopee/planilhas/outputs/final.xlsx')

marcas = df['Marca'].unique().tolist()

marcas

In [ ]:
import re
import pandas as pd

# Certifique-se que o nome da coluna é exatamente este:
col = "Descrição"

df = pd.read_excel('/home/lucas-silva/auto_shopee/planilhas/outputs/final.xlsx')

# Lista de categorias pesadas
heavy_keywords = [
    r"\bTELHA\b", r"\bCUMEEIRA\b",
    r"\bPORTA\b", r"GABINETE", r"\bVASO\b", r"LAVAT[ÓO]RIO", r"\bPIA\b", r"\bTANQUE\b",
    r"CAIXA D'?[ÁA]GUA",r"CANTON", r"EXTENSOR", r"BACIA",r"VIAPLUS", r"VEDAPREN",
    r"TUBO", r"CONEX[ÃA]O",r"CANAL", r"LONA", r"TECPLUS",r"SIKATOP", r"CORRENTE",
    r"CIMENTO", r"ARGAMASSA", r"MASSA CORRIDA", r"MASSA ACR[IÍ]LICA",
    r"TINTA", r"VEDATOP", r"\bCALHA\b",r"ELETRODUTO",r"VAR[ÃA]O",
    r"\bPÁ\b", r"\bENXADA\b", r"CAVADEIRA", r"\bFORCADO\b", r"R[EÉ]GUA ALUM[IÍ]NIO",
    r"CARRINHO", r"\bESCADA\b", r"VARAL DE CH[ÃA]O",r"MANTA", r"DAGUA",r"COLUNA", r"\bLAV\b",r"TELA"
]

# Falsos positivos que devem ser ignorados mesmo contendo palavras pesadas
exclude_keywords = [
    "PARAFUSO P/ TELHA", "PARAF","FECH","SOLDA","LIGACAO","VALV","ANEL","SERRA","BOCAL",
    "ABRAC", "CADEADO ", "GRELHA", "TORN","DESEMP","RALINHO","PINO", "BOLSA","ADAP","PLASTICO",
    "TRAVA", "BOLSA","LIMA", "RALINHO","FITA","CABO FLEX", "CABO REDE", "PATCH CORD", "COAXIAL",
    "CABO PP","BATEDOR","BALDE","ABRAC","SELATRINCA","FIXADOR","PILHA","FRISO","MOVEIS","REFIL",
    "VIDRO", "PRATELEIRA","FRANC", "PRESSURIZADOR", "FILTRO","PENEIRA","LUVA","CALCO","DISCO",
    "SUP", "BATEADOR","MISTURADOR","SAIDA", "CABECEIRA", "PNEU P/ CARRINHO"
]

pesados_dict = {}

for idx, desc in df[col].items():
    d = str(desc).upper()

    # Exclusões diretas
    if any(ex in d for ex in exclude_keywords):
        continue

        # Massa corrida leve (não entra)
    if ("MASSA CORRIDA" in d or "MASSA ACR" in d) and any(x in d for x in ["1KG", "900G", "500G", "3.6KG"]):
        continue

    # Tinta só é pesada se 18L ou + (seu padrão)
    if "TINTA" in d and not any(x in d for x in ["15","18L", "20L"]):
        continue


    # Checagem principal
    if any(re.search(k, d) for k in heavy_keywords):
        pesados_dict[idx] = desc

# Criar dataframes finais
df_pesados = df.loc[df.index.isin(pesados_dict.keys())].copy()
df_restante = df.loc[~df.index.isin(pesados_dict.keys())].copy()


print("Itens pesados:", len(df_pesados))
print("Itens restantes:", len(df_restante))

df_restante.to_excel('grandes.xlsx')

df_pesados

In [ ]:
import pandas as pd

df = pd.read_excel('/home/lucas-silva/auto_shopee/planilhas/input/shopee_template.xlsx',
                   sheet_name="Modelo")

colunas = df.columns.tolist()

colunas

In [ ]:
import pandas as pd

df = pd.read_excel('grandes.xlsx')


colunas = df.columns.tolist()

colunas

In [2]:
import pandas as pd
import cloudinary
import cloudinary.uploader
import os

# -----------------------------------------------------
# CONFIGURAÇÃO DO CLOUDINARY
# -----------------------------------------------------
cloudinary.config(
    cloud_name="dbpq32fiq",
    api_key="728699121312431",
    api_secret="pzmdaYjFR0lDCwJhpFAXNJ0OV9Y"
)

# -----------------------------------------------------
# ARQUIVO E NOMES DAS COLUNAS
# -----------------------------------------------------
ARQUIVO = "/home/lucas-silva/auto_shopee/final_com_urls.xlsx"
COL_URL = "Url Imagem"
COL_CODIGO = "Codigo Produto"

# -----------------------------------------------------
# 1 - CARREGAR ARQUIVO
# -----------------------------------------------------
df = pd.read_excel(ARQUIVO)

# Criar nova coluna para URL final
df["Url Cloudinary"] = ""


# -----------------------------------------------------
# 2 - PROCESSAR LINHA POR LINHA
# -----------------------------------------------------
for i, row in df.iterrows():
    url_imagem = row[COL_URL]
    codigo = str(row[COL_CODIGO]).strip()

    print(f"\nProcessando linha {i+1}/{len(df)}") # pyright: ignore[reportOperatorIssue]
    print("Código:", codigo)
    print("Imagem:", url_imagem)

    # Pular URLs vazias
    if pd.isna(url_imagem):
        print("❌ URL vazia! Pulando...")
        df.at[i, "Url Cloudinary"] = "URL VAZIA" # type: ignore
        continue

    # Pasta baseada no código do produto
    pasta_cloud = f"shopee/produtos/{codigo}"

    # Nome do arquivo no Cloudinary
    nome_arquivo = codigo  # vai virar codigo.jpg ou png

    try:
        result = cloudinary.uploader.upload(
            url_imagem,
            folder=pasta_cloud,
            public_id=nome_arquivo,  # renomeia a imagem
            overwrite=True,          # substitui se já existir
            use_filename=False,
            unique_filename=False
        )

        nova_url = result["secure_url"]
        print("✔ Enviado:", nova_url)

        df.at[i, "Url Cloudinary"] = nova_url # type: ignore

    except Exception as e:
        print("❌ Erro no upload:", e)
        df.at[i, "Url Cloudinary"] = "ERRO" # type: ignore


# -----------------------------------------------------
# 3 - SALVAR ARQUIVO ATUALIZADO
# -----------------------------------------------------
df.to_excel("final_urls_cloudinary.xlsx", index=False)

print("\n------------------------------------------")
print("PROCESSO FINALIZADO!")
print("Arquivo salvo como: final_urls_cloudinary.xlsx")
print("------------------------------------------")



Processando linha 1/3809
Código: 186124
Imagem: https://cdn.construja.com.br/produtos/186124/186124.jpg
✔ Enviado: https://res.cloudinary.com/dbpq32fiq/image/upload/v1763507049/shopee/produtos/186124/186124.jpg

Processando linha 2/3809
Código: 170188
Imagem: https://cdn.construja.com.br/produtos/170188/170188.jpg
✔ Enviado: https://res.cloudinary.com/dbpq32fiq/image/upload/v1763507050/shopee/produtos/170188/170188.jpg

Processando linha 3/3809
Código: 156143
Imagem: https://cdn.construja.com.br/produtos/156143/156143.jpg
✔ Enviado: https://res.cloudinary.com/dbpq32fiq/image/upload/v1763507052/shopee/produtos/156143/156143.jpg

Processando linha 4/3809
Código: 170184
Imagem: https://cdn.construja.com.br/produtos/170184/170184.jpg
✔ Enviado: https://res.cloudinary.com/dbpq32fiq/image/upload/v1763507053/shopee/produtos/170184/170184.jpg

Processando linha 5/3809
Código: 170124
Imagem: https://cdn.construja.com.br/produtos/170124/170124.jpg
✔ Enviado: https://res.cloudinary.com/dbpq32fiq